#Transformers и Datasets

In [3]:
! pip install datasets transformers[sentencepiece] sacrebleu
! pip install accelerate==0.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 4.5 MB/s eta 0:00:00


In [4]:
!apt install git-lfs
!git config --global user.email "zagrebelnaya81@gmail.com"
!git config --global user.name "Irina Zagrebelnaya"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
import transformers

print(transformers.__version__)

4.31.0


Буду использовать набор данных WMT

In [7]:
model_checkpoint = "agemagician/mlong-t5-tglobal-base"

Здесь я выбрала agemagician/mlong-t5-tglobal-base

#Загрузка набора данных

Используеться англо-немецкая часть набора данных WMT.

In [8]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "de-en")
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

<ipython-input-8-4995781440e0>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


dataset — это , DatasetDict который содержит один ключ для обучения, проверки и набора тестов

In [9]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4548885
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2169
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})

Посмотрим как выглядят рандомно выбранные данные

In [10]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Не могу выбрать."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["test"])

,translation
0,"{'de': '2013 hatten verspätete Lieferungen den Paketdiensten harsche Kritik eingebracht.', 'en': 'In 2013 there was harsh criticism of late deliveries by the parcel service.'}"
1,"{'de': 'Er sagte: ""E-Zigaretten sind wesentlich weniger schädlich als Rauchen.""', 'en': 'He said: ""E-cigarettes are significantly less harmful than smoking.""'}"
2,"{'de': '""Es ist unmöglich, dass jeder nach Europa kommen kann"", sagte er in seiner Rede.', 'en': '""It it's impossible for everyone to come to Europe,"" he said in his speech.'}"
3,"{'de': 'Zwei der drei beteiligten Lastwagen hatten vor einer Dauerbaustelle in Richtung Basel gebremst.', 'en': 'Two of the three lorries involved had braked approaching long-term roadworks in the direction of Basel.'}"
4,"{'de': 'So könnte auf die Grunderwerbssteuer verzichtet werden und die Grundsteuer für geförderte Wohnungen befristet ausgesetzt werden.', 'en': 'For example land transfer tax could be waived and the land tax for sponsored housing temporarily suspended.'}"


# Предварительная оброботка данных

токенизируем входные данные для перевода их в формат, ожидаемый моделью, а также генерируют другие входные данные, которые модель требует.
Мы получаем токенизатор, соответствующий архитектуре модели, которую мы хотим использовать,и загружаем словарный запас, используемый при предварительном обучении

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Нам нужно установить исходный и целевой языки (чтобы тексты были предварительно обработаны должным образом)

In [12]:
tokenizer.src_lang = "en-XX"
tokenizer.tgt_lang = "de-DE"

In [13]:
tokenizer(["HuggingFace’s T5 model offers a powerful and flexible solution for translation tasks.", "By following this guide, you can harness its capabilities to translate text between various languages."])

{'input_ids': [[7718, 38695, 129144, 308, 281, 427, 284, 2539, 10303, 289, 44731, 333, 30807, 10599, 341, 52281, 84249, 274, 1], [1911, 10151, 782, 9309, 275, 534, 774, 455, 5794, 2316, 130356, 315, 122018, 5088, 5002, 14579, 101996, 274, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

Функция, которой скармливаем их tokenizer аргументу truncation=True.Входные данные, длина которых превышает возможности выбранной модели, будут усечены до максимальной длины

In [14]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [15]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "de"

def prepare_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
tokenized_datasets = raw_datasets.map(prepare_function, batched=True)

Map:   0%|          | 0/4548885 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2169 [00:00<?, ? examples/s]

Map:   0%|          | 0/2999 [00:00<?, ? examples/s]

# Тонкая настройка модели

In [17]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [18]:
batch_size = 3
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
)

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)


    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [21]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/zagrebelnaya81/mlong-t5-tglobal-base-finetuned-en-to-de into local empty directory.


In [22]:
#Насраиваем модель
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Epoch,Training Loss,Validation Loss


Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1539 in train                    │
│                                                                                                  │
│   1536 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1537 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1538 │   │   )                                                                                 │
│ ❱ 1539 │   │   return inner_training_loop(                                                       │
│   1540 │   │   │   args=args,                                                                    │
│   1541 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1542 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1809 in _inner_training_loop     │
│                                                                                                  │
│   1806 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1807 │   │   │   │                                                                             │
│   1808 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1809 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1810 │   │   │   │                                                                             │
│   1811 │   │   │   │   if (                                                                      │
│   1812 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2665 in training_step            │
│                                                                                                  │
│   2662 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2663 │   │   │   │   scaled_loss.backward()                                                    │
│   2664 │   │   else:                                                                             │
│ ❱ 2665 │   │   │   self.accelerator.backward(loss)                                               │
│   2666 │   │                                                                                     │
│   2667 │   │   return loss.detach() / self.args.gradient_accumulation_steps                      │
│   2668                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:1821 in backward               │
│                                                                                                  │
│   1818 │   │   elif self.scaler is not None:                                                     │
│   1819 │   │   │   self.scaler.scale(loss).backward(**kwargs)                                    │
│   1820 │   │   else:                                                                             │
│ ❱ 1821 │   │   │   loss.backward(**kwargs)                                                       │
│   1822 │                                                   

In [23]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/2.21G [00:00<?, ?B/s]

To https://huggingface.co/zagrebelnaya81/mlong-t5-tglobal-base-finetuned-en-to-de
   c494159..4875460  main -> main

   c494159..4875460  main -> main

To https://huggingface.co/zagrebelnaya81/mlong-t5-tglobal-base-finetuned-en-to-de
   4875460..cd34a0d  main -> main

   4875460..cd34a0d  main -> main



'https://huggingface.co/zagrebelnaya81/mlong-t5-tglobal-base-finetuned-en-to-de/commit/48754600810fe834b8137e48bb1e6965cab81443'

In [26]:
from transformers import pipeline

model_checkpoint = "zagrebelnaya81/mlong-t5-tglobal-base-finetuned-en-to-de"
translator = pipeline("translation", model=model_checkpoint)
translator("You have 1 active session.?")

[{'translation_text': 'Sie haben 1 aktive Session.'}]

Исходя из того что за 10 часов модель натринеровалась всего на около 20000 из 1500000, перевод достаточно сносный.)
просто я поняла что для плной тренировки модели на датасете который я выбрала нужно очень много времени выгрузила то что имела на данный момент и проверила как она переводит. Хотя во многих фразах били небоьлшие неточсности, все же общийсмысл перевода удовлетвориельный